In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import gensim
import random
from gensim.models import Word2Vec
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from matplotlib import font_manager, rc
from pandas import DataFrame as df
import pickle

# word2vec

아이템 임베딩을 만들어서 유사도 구하는 방법

## 전처리

In [6]:
movies = pd.read_csv('ml-25m/movies.csv')
ratings = pd.read_csv('ml-25m/ratings.csv')

In [6]:
movieId_to_name = pd.Series(movies.title.values, index = movies.movieId.values).to_dict()
name_to_movieId = pd.Series(movies.movieId.values, index = movies.title).to_dict()

In [36]:
print(ratings[ratings['userId'].isin(['1'])])

    userId movieId  rating   timestamp  liked
0        1     296     5.0  1147880044      1
1        1     306     3.5  1147868817      0
2        1     307     5.0  1147868828      1
3        1     665     5.0  1147878820      1
4        1     899     3.5  1147868510      0
..     ...     ...     ...         ...    ...
65       1   27193     3.0  1147879774      0
66       1   27266     4.5  1147879365      1
67       1   27721     3.0  1147869115      0
68       1   31956     3.5  1147877610      0
69       1   32591     5.0  1147879538      1

[70 rows x 5 columns]


In [44]:
len(movieId_to_name)

62423

ratings를 train과 test로 나누기

In [8]:
#ratings_train, ratings_test = train_test_split(ratings)
#ratings_train.to_pickle("ratings_train.pkl")
#ratings_test.to_pickle("ratings_test.pkl")

In [4]:
with open('ratings_train.pkl', 'rb') as f:
    ratings_train = pickle.load(f)

In [5]:
"""
ratings_train['liked'] = np.where(ratings_train['rating']>=4, 1, 0)
ratings_train['movieId'] = ratings_train['movieId'].astype('str')
gp_user_like = ratings_train.groupby(['liked', 'userId'])
"""
ratings['liked'] = np.where(ratings['rating']>=4, 1, 0)
ratings['movieId'] = ratings['movieId'].astype('str')
gp_user_like = ratings.groupby(['liked', 'userId'])

NameError: name 'ratings' is not defined

In [11]:
# 유저 n이 좋아한 영화 => positive example
# 유저 n이 싫어하는 영화 별로 그룹, 좋아하는 영화 별로 그룹핑
splitted_movies = [gp_user_like.get_group(gp)['movieId'].tolist() for gp in gp_user_like.groups]

In [12]:
for movie_list in splitted_movies:
    random.shuffle(movie_list)

In [1]:
splitted_movies

NameError: name 'splitted_movies' is not defined

In [13]:
print("전처리 완료")

전처리 완료


In [ ]:
model = Word2Vec(sentences = splitted_movies,
                 iter = 20, # epoch
                 size = 100, # music dataset 파라미터 사용
                 sg = 1, # skip-gram: 1
                 hs = 0, # Set to 0, as we are applying negative sampling.
                 negative = 5, # negative sampling
                 window = 9999999)
model.save('item2vec_embeddings')

## 모델 로드

In [37]:
model = Word2Vec.load('item2vec_embeddings')

In [77]:
items = list(model.wv.vocab)
max(items)

'99999'

In [84]:
def similar_basket(movieId, threshold = 0.9):
    sim_basket = [item for item in model.wv.most_similar_cosmul(movieId) if item[1] > threshold]
    sbasket = []
    for i in sim_basket:
        mid = int(i[0])
        mname = movieId_to_name[mid]
        sbasket.append([mname, i[1]])
    
    return sbasket

In [85]:
sbasket = similar_basket('99999', 0)
print(movieId_to_name[99999])
print(sbasket)

Texas Across the River (1966)
[['Cowboy and the Lady, The (1938)', 0.8801010847091675], ['Sun Valley Serenade (1941)', 0.8744916915893555], ['Comrade X (1940)', 0.8647785186767578], ['Up Periscope (1959)', 0.8626752495765686], ['Seven Little Foys, The (1955)', 0.8606352806091309], ['Jonathan Livingston Seagull (1973)', 0.8536442518234253], ['Damn the Defiant! (H.M.S. Defiant) (1962)', 0.8529667854309082], ['Wee Willie Winkie (1937)', 0.852799117565155], ['Man Between, The (1953)', 0.8445892333984375], ['Fate Is the Hunter (1964)', 0.8431283831596375]]


## 문제점

1. user-item 세트, CF의 문제점을 해결하지 못함
2. 임베딩으로 바스켓 추천 어떻게 할 것인가... 유사도 측정밖에 생각이 안남...ㅠ
3. 롱테일 현상(소수의 영화만 평점을 많이 받고 나머지 영화는 평점 거의 받지 못함)